In [13]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *




In [14]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")

We're going to exclude a few subjects because their data is not useable.

In [15]:
test_train_df_raw['IncludeInSST'] = True
test_train_df_raw.loc[test_train_df_raw.sub_label=='DEV001','IncludeInSST']=False

test_train_df = test_train_df_raw[test_train_df_raw.IncludeInSST==True]

In [19]:
dataset_name = 'conditions'
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_' + dataset_name + '_2subs.pkl'

with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_2subs.pkl'

In [17]:
Brain_Data_allsubs.X

,onset,duration,trial_type,subject,wave,beta
256,0.00000,2.25834,correct-stop,DEV005,wave1,beta_0001.nii
257,2.25834,0.50000,ITI,DEV005,wave1,beta_0002.nii
258,2.75834,0.40082,correct-go,DEV005,wave1,beta_0003.nii
259,4.76390,0.75000,ITI,DEV005,wave1,beta_0004.nii
260,5.51390,0.66191,correct-go,DEV005,wave1,beta_0005.nii
...,...,...,...,...,...,...
2555,412.82572,1.75000,ITI,DEV013,wave1,beta_0252.nii
2556,414.57572,0.56873,correct-go,DEV013,wave1,beta_0253.nii
2557,417.08682,0.75000,ITI,DEV013,wave1,beta_0254.nii
2558,417.83682,1.50000,failed-stop,DEV013,wave1,beta_0255.nii


In [6]:
def response_transform_func(X):
    return(X.trial_type)

In [7]:
from nilearn.decoding import DecoderRegressor, Decoder

In [8]:
response_transform_func(Brain_Data_allsubs.X).value_counts()

ITI             768
correct-go      551
failed-stop     145
correct-stop     47
failed-go        25
Name: trial_type, dtype: int64

In [9]:
first_6_subjs = test_train_df[test_train_df.SplitGroup=='Train'][0:6]
first_6_subjs

,sub_label,SplitGroup,IncludeInSST
2,DEV005,Train,True
3,DEV006,Train,True
5,DEV009,Train,True
6,DEV010,Train,True
7,DEV012,Train,True
8,DEV013,Train,True


In [10]:
X_df=Brain_Data_allsubs.X

In [11]:
for subj in first_6_subjs.sub_label:
    print(subj)
    print(
        response_transform_func(X_df[X_df.subject==subj]).value_counts()
    )

DEV005
ITI             128
correct-go       80
failed-stop      27
failed-go        16
correct-stop      5
Name: trial_type, dtype: int64
DEV006
ITI             128
correct-go       92
correct-stop     18
failed-stop      14
failed-go         4
Name: trial_type, dtype: int64
DEV009
ITI            128
correct-go      96
failed-stop     32
Name: trial_type, dtype: int64
DEV010
ITI             128
correct-go       94
failed-stop      19
correct-stop     13
failed-go         2
Name: trial_type, dtype: int64
DEV012
ITI            128
correct-go      95
failed-stop     32
failed-go        1
Name: trial_type, dtype: int64
DEV013
ITI             128
correct-go       94
failed-stop      21
correct-stop     11
failed-go         2
Name: trial_type, dtype: int64


In [12]:
script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'
data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/"
# HRF 2s
dataset_name = 'betaseries'
apply_loocv_and_save(
    results_filepath=data_path + "SST/train_test_results_" + dataset_name + "_6subs.pkl",
    brain_data_filepath = data_path + 'SST/Brain_Data_' + dataset_name + '_6subs.pkl',
    train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv",
    subjs_to_use = 6,
    response_transform_func = response_transform_func,
    decoderConstructor=Decoder #for classification
)


checked for intersection and no intersection between the brain data and the subjects was found.
there were 6 subjects overlapping between the subjects marked for train data and the training dump file itself.
ITI             768
correct-go      551
failed-stop     145
correct-stop     47
failed-go        25
Name: trial_type, dtype: int64
ITI             768
correct-go      551
failed-stop     145
correct-stop     47
failed-go        25
Name: trial_type, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 138
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    1536
Name: trial_type, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


1536
1536
using 6 subjects
starting LeaveOneOut
finished preprocessing
Groups are the same.
fold 1 of 6
In order to test on a training group of 5 items, holding out the following subjects:['DEV013']. prepping fold data.... fitting.... 10.3 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

So what do we actually want to measure here?

 - For the Response to Failure study we want error related negativity. That means we probably want the period immediately *following* an action. Ideally that means we create another set of betas that measure time following the error, and we relate that to post-error follows.
 - For the TESQ-E Self Control study we want the measure of inhibition. Probably we want to classify stop signal reaction time given people's behavioral responses. We will need to _control_ for the actual moment of the reaction. Mainly, we are interested in different processing techniques that lead to faster or slower stop signal reaction times. Need to look at those results and how this task actually relates; but we don't have those results yet because we're still waiting on the process Bryan is doing. But it is unclear to me whehter stop signal reaction time will actually relate to self control. Does faster stop signal reaction time mean better performance? or is it _slower_ stop signal reaction time? I don't know.

